# Download Sentinel-1 RTC for each ASO raster
Given an ASO raster, find and download 1) the most proximate in time S1 RTC product 2) a "snow-off" RTC product from the preceeding summer. 

In [1]:
# based on exmaples from
# https://planetarycomputer.microsoft.com/docs/tutorials/cloudless-mosaic-sentinel2/
# https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a#Example-Notebook
from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer
import glob
import rioxarray as rxr
import re
import datetime
import pandas as pd
from shapely.geometry import box
import odc.stac
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import rasterio as rio
import traceback

In [2]:
def rtc_for_aso_snowon(aso_raster_fn):
    time = pd.to_datetime(re.search("(\d{4}\d{2}\d{2})", aso_raster_fn).group())
    week_before = (time - datetime.timedelta(weeks=2)).strftime('%Y-%m-%d')
    week_after = (time + datetime.timedelta(weeks=2)).strftime('%Y-%m-%d')
    time_of_interest = f'{week_before}/{week_after}'
    
    aso_raster = rxr.open_rasterio(aso_raster_fn).squeeze()
    aso_raster = aso_raster.where(aso_raster>=0, drop=True)
    bounds_latlon = box(*aso_raster.rio.transform_bounds("EPSG:4326"))
    
    catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace)

    search = catalog.search(
        collections=["sentinel-1-rtc"],
        intersects=bounds_latlon,
        datetime=time_of_interest)
    
    # Check how many items were returned
    items = search.item_collection()
    
    rtc_stac = odc.stac.load(items,chunks={"x": 2048, "y": 2048},resolution=50, groupby='sat:absolute_orbit')
    print(f"Returned {len(rtc_stac.time)} acquisitions")
    rtc_stac_clipped = rtc_stac.rio.clip_box(*bounds_latlon.bounds,crs="EPSG:4326")
    
    rel_orbits = [scene.properties['sat:relative_orbit'] for scene in items.items]
    ac_times = [scene.properties['datetime'] for scene in items.items]
    ac_times = [np.datetime64(item) for item in ac_times]
    
    # clip to ASO extent
    rtc_stac_clipped = rtc_stac_clipped.rio.reproject_match(aso_raster, resampling=rio.enums.Resampling.bilinear)

    # limit to morning acquisitions
    rtc_ds = rtc_stac_clipped.where(rtc_stac_clipped.time.dt.hour > 11, drop=True)
    if 'vv' not in list(rtc_ds.keys()) or 'vh' not in list(rtc_ds.keys()):
        print('missing polarization')
        return None
    
    if len(rtc_ds.time) == 0:
        print('no morning acquisitions')
        return None
        
    # calculate percent vh coverage of each acquisition
    perc_cover = (rtc_ds.vh.where(aso_raster >= 0) > 0).sum(dim=['x', 'y'])/(rtc_ds.vh.where(aso_raster >= 0) >= -1000000000).sum(dim=['x', 'y'])
    
    # if multiple with full coverage, grab nearest in time with full coverage
    if perc_cover.values.tolist().count(1) > 1:
        print('total snow-on coverage available')
        rtc_ds = rtc_ds.where(perc_cover == 1, drop=True).sortby('time')
        rtc_ds = rtc_ds.sel(time=time, method='nearest')

    # exit if no rasters have good vh coverage
    elif perc_cover.max() < 0.01:
        print('max vh coverage is < 1%--recommend skipping ASO raster')
        return None

    # otherwise, grab max coverage 
    else:
        if perc_cover.max() == 1:
            print('total snow-on coverage available')
        else: 
            print(f'{perc_cover.max().item()} snow-on coverage')
        rtc_ds = rtc_ds.sel(time=perc_cover.idxmax())

    # get relative orbit of scene
    rel_orbit = rel_orbits[ac_times.index(rtc_ds.time)]
    
    # mask negative areas
    rtc_ds = rtc_ds.where(rtc_ds.vh > 0)
    rtc_ds = rtc_ds.where(rtc_ds.vv > 0)
    
    rtc_ds.to_netcdf(f'../data/S1_rtc/S1_snow-on_{rtc_ds.time.dt.strftime("%Y%m%d").item()}_for_{aso_raster_fn.split("/")[-1][:-4]}.nc')
    
    return rel_orbit

In [3]:
def rtc_for_aso_snowoff(aso_raster_fn, rel_orbit):
    year = pd.to_datetime(re.search("(\d{4}\d{2}\d{2})", aso_raster_fn).group()).year
    time = pd.to_datetime(f'{year-1}0910')
    week_before = (time - datetime.timedelta(weeks=2)).strftime('%Y-%m-%d')
    week_after = (time + datetime.timedelta(weeks=2)).strftime('%Y-%m-%d')
    time_of_interest = f'{week_before}/{week_after}'

    aso_raster = rxr.open_rasterio(aso_raster_fn).squeeze()
    aso_raster = aso_raster.where(aso_raster>=0, drop=True)
    bounds_latlon = box(*aso_raster.rio.transform_bounds("EPSG:4326"))

    catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace)

    search = catalog.search(
        collections=["sentinel-1-rtc"],
        intersects=bounds_latlon,
        datetime=time_of_interest)

    # Check how many items were returned
    items = search.item_collection()

    rel_orbits = [scene.properties['sat:relative_orbit'] for scene in items.items]
    ac_times = [scene.properties['datetime'] for scene in items.items]
    ac_times = [np.datetime64(item) for item in ac_times]

    rtc_stac = odc.stac.load(items,chunks={"x": 2048, "y": 2048},resolution=50, groupby='sat:absolute_orbit')
    print(f"Returned {len(rtc_stac.time)} acquisitions")
    rtc_stac_clipped = rtc_stac.rio.clip_box(*bounds_latlon.bounds,crs="EPSG:4326")

    orbit_dict = {}
    for i, orbit in enumerate(rel_orbits):
        if orbit not in orbit_dict.keys():
            orbit_dict[orbit] = [ac_times[i]]
        else:
            orbit_dict[orbit].append(ac_times[i])

    if rel_orbit not in orbit_dict.keys():
        print('no acquisitons from same orbit, skipping')
        return

    rtc_stac_clipped = rtc_stac_clipped.where(rtc_stac_clipped.time.isin(orbit_dict[rel_orbit]), drop=True)

    # clip to ASO extent
    rtc_stac_clipped = rtc_stac_clipped.rio.reproject_match(aso_raster, resampling=rio.enums.Resampling.bilinear)

    # limit to morning acquisitions
    rtc_ds = rtc_stac_clipped.where(rtc_stac_clipped.time.dt.hour > 11, drop=True)
    if 'vv' not in list(rtc_ds.keys()) or 'vh' not in list(rtc_ds.keys()):
            print('missing polarization, skipping')
            return
    
    if len(rtc_ds.time) == 0:
        print('no morning acquisitions')
        return None

    # calculate percent vh coverage of each acquisition
    perc_cover = (rtc_ds.vh.where(aso_raster >= 0) > 0).sum(dim=['x', 'y'])/(rtc_ds.vh.where(aso_raster >= 0) >= -1000000000).sum(dim=['x', 'y'])

    # if multiple with full coverage, grab nearest in time with full coverage
    if perc_cover.values.tolist().count(1) > 1:
        print('total snow-off coverage available')
        rtc_ds = rtc_ds.where(perc_cover == 1, drop=True).sortby('time')
        rtc_ds = rtc_ds.sel(time=time, method='nearest')

    # exit if no rasters have good vh coverage
    elif perc_cover.max() < 0.01:
        print('max vh coverage is < 1%--recommend skipping ASO raster')
        return

    # otherwise, grab max coverage 
    else:
        if perc_cover.max() == 1:
            print('total snow-off coverage available')
        else: 
            print(f'{perc_cover.max().item()} snow-off coverage')
        rtc_ds = rtc_ds.sel(time=perc_cover.idxmax())
    
    # mask negative areas
    rtc_ds = rtc_ds.where(rtc_ds.vh > 0)
    rtc_ds = rtc_ds.where(rtc_ds.vv > 0)
    
    rtc_ds.to_netcdf(f'../data/S1_rtc/S1_snow-off_{rtc_ds.time.dt.strftime("%Y%m%d").item()}_for_{aso_raster_fn.split("/")[-1][:-4]}.nc')

In [4]:
def rtc_for_aso_all(dir_path):
    raster_paths = glob.glob(f'{dir_path}/*/ASO_50M_SD*.tif')
    for i, path in enumerate(raster_paths):
        print(f'----\nworking on {path.split("/")[-1]}, {i+1}/{len(raster_paths)}\n----')
        
        try:
            relative_orbit = rtc_for_aso_snowon(path)
            if relative_orbit == None:
                continue
            rtc_for_aso_snowoff(path, relative_orbit)
        except Exception as exc:
            print(traceback.format_exc())
            print(exc)
            print('encountered error, skipping')

In [5]:
dir_path = '/home/jovyan/deep-snow/data/ASO/ASO_50m_SD_cleaned'
test = rtc_for_aso_all(dir_path)

----
working on ASO_50M_SD_BlueRiver_20230529_clean.tif, 1/252
----
Returned 4 acquisitions


/tmp/ipykernel_2688/2824340958.py:29: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  ac_times = [np.datetime64(item) for item in ac_times]
/srv/conda/envs/notebook/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/srv/conda/envs/notebook/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


total snow-on coverage available
Returned 5 acquisitions
total snow-off coverage available
----
working on ASO_50M_SD_Animas_20210419_clean.tif, 2/252
----
Returned 7 acquisitions
total snow-on coverage available
Returned 7 acquisitions
total snow-off coverage available
----
working on ASO_50M_SD_EastRiver_20230523_clean.tif, 3/252
----
Returned 8 acquisitions
total snow-on coverage available
Returned 8 acquisitions
total snow-off coverage available
----
working on ASO_50M_SD_EastRiver_20220518_clean.tif, 4/252
----
Returned 5 acquisitions
total snow-on coverage available
Returned 9 acquisitions
total snow-off coverage available
----
working on ASO_50M_SD_SouthPlatte_20230416_clean.tif, 5/252
----
Returned 9 acquisitions
total snow-on coverage available
Returned 10 acquisitions
total snow-off coverage available
----
working on ASO_50M_SD_USCOCM_20190407_clean.tif, 6/252
----
Returned 6 acquisitions
total snow-on coverage available
Returned 10 acquisitions
total snow-off coverage availa